Importing libraries

In [18]:
!pip install ta

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29496 sha256=e804f10824a4aa5d700618f2660a955e295802e0ec35b7765f1efd835bae0dd8
  Stored in directory: c:\users\rdine\appdata\local\pip\cache\wheels\e3\3a\ee\4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  DEPRECATION: Building 'ta' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'ta'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [55]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
from xgboost.callback import EarlyStopping

In [48]:
df = pd.read_csv("btc_1d_data_2018_to_2025.csv")

df.head()

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
0,2018-01-01,13715.65,13818.55,12750.00,13380.00,8609.915844,2018-01-01 23:59:59.999,1.147997e+08,105595,3961.938946,5.280975e+07,0
1,2018-01-02,13382.16,15473.49,12890.02,14675.11,20078.092111,2018-01-02 23:59:59.999,2.797171e+08,177728,11346.326739,1.580801e+08,0
2,2018-01-03,14690.00,15307.56,14150.00,14919.51,15905.667639,2018-01-03 23:59:59.999,2.361169e+08,162787,8994.953566,1.335873e+08,0
3,2018-01-04,14919.51,15280.00,13918.04,15059.54,21329.649574,2018-01-04 23:59:59.999,3.127816e+08,170310,12680.812951,1.861168e+08,0
4,2018-01-05,15059.56,17176.24,14600.00,16960.39,23251.491125,2018-01-05 23:59:59.999,3.693220e+08,192969,13346.622293,2.118299e+08,0


In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2705 entries, 0 to 2704
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Open time                     2705 non-null   object 
 1   Open                          2705 non-null   float64
 2   High                          2705 non-null   float64
 3   Low                           2705 non-null   float64
 4   Close                         2705 non-null   float64
 5   Volume                        2705 non-null   float64
 6   Close time                    2705 non-null   object 
 7   Quote asset volume            2705 non-null   float64
 8   Number of trades              2705 non-null   int64  
 9   Taker buy base asset volume   2705 non-null   float64
 10  Taker buy quote asset volume  2705 non-null   float64
 11  Ignore                        2705 non-null   int64  
dtypes: float64(8), int64(2), object(2)
memory usage: 253.7+ KB


In [50]:
df.columns.to_list()

['Open time',
 'Open',
 'High',
 'Low',
 'Close',
 'Volume',
 'Close time',
 'Quote asset volume',
 'Number of trades',
 'Taker buy base asset volume',
 'Taker buy quote asset volume',
 'Ignore']

Data preprocessing

In [51]:
# Cleaning column name
df.columns = df.columns.str.strip()

In [52]:
df.isna()

,Open time,Open,High,Low,Close,Volume,Close time,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume,Ignore
0,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
2700,False,False,False,False,False,False,False,False,False,False,False,False
2701,False,False,False,False,False,False,False,False,False,False,False,False
2702,False,False,False,False,False,False,False,False,False,False,False,False
2703,False,False,False,False,False,False,False,False,False,False,False,False


Parse datetime

In [59]:
df['Open time'] = pd.to_datetime(df['Open time'])
df.sort_values('Open time', inplace=True)
df.reset_index(drop=True, inplace=True)

df['target_t+1'] = df['Close'].shift(-1)
LAG_DAYS = 7
for lag in range(1, LAG_DAYS + 1):
    df[f'Close_t-{lag}'] = df['Close'].shift(lag)
    df[f'Volume_t-{lag}'] = df['Volume'].shift(lag)
    df[f'Open_t-{lag}'] = df['Open'].shift(lag)
    df[f'High_t-{lag}'] = df['High'].shift(lag)
    df[f'Low_t-{lag}'] = df['Low'].shift(lag)

df['rolling_close_mean_7'] = df['Close'].rolling(window=7).mean()
df['rolling_close_std_7'] = df['Close'].rolling(window=7).std()
df['rolling_volume_mean_7'] = df['Volume'].rolling(window=7).mean()

df['dayofweek'] = df['Open time'].dt.dayofweek
df['month'] = df['Open time'].dt.month

# Drop rows with any NaNs (from shifting & rolling)
df_model = df.dropna().copy()

# Drop unused columns
drop_cols = ['Open time', 'Close time', 'Ignore', 'target_t+1']
features = df_model.drop(columns=drop_cols + ['target_t+1']).columns.tolist()

# Define features and target
X = df_model[features]
y = df_model['target_t+1']

split_idx = int(len(X) * 0.8)
X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

model = XGBRegressor(
    n_estimators=300,
    max_depth=5,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(X_train, y_train, 
          eval_set=[(X_test, y_test)],
          eval_metric="rmse",
          callbacks=[EarlyStopping(rounds=20)],
          verbose=False)

y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print(f"RMSE: {rmse:.2f}")
print(f"MAE: {mae:.2f}")

# Plot predictions
plt.figure(figsize=(14, 5))
plt.plot(y_test.values, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.title("BTC Close Price Prediction (Next Day)")
plt.legend()
plt.show()


TypeError: XGBModel.fit() got an unexpected keyword argument 'disable_default_eval_metric'

Technical indicators

In [57]:
import xgboost
print(xgboost.__version__)


3.0.0


Target variables

Data visualization